In [2]:
import numpy as np
from scipy import stats as st
from matplotlib import pyplot as plt

%matplotlib inline

In [3]:
n = 100
teta = 2

Генерация выборки

In [4]:
data = np.power(1 - np.random.rand(n), 1 / (1 - teta))
data

array([ 8.3778676 ,  6.87314297,  1.1202374 ,  8.50486674,  1.32730026,
        4.42645854,  1.42170599,  5.2324678 ,  1.74306169,  2.16609378,
        1.17210948,  1.97163798,  4.44857706,  1.46243975, 22.08818639,
        3.06238501,  1.03944874,  2.18801724,  2.11177913,  1.78535482,
        3.2492383 ,  1.55006718,  1.28409317,  1.09354812,  8.9863811 ,
        1.52025285,  4.60996848,  3.1349956 ,  8.99742391,  3.83247161,
        1.08058727,  1.23726687,  5.39459648,  1.04436313,  9.69015046,
        1.18709475,  1.1545741 ,  5.29152606,  2.32568433,  1.36687688,
        1.66081735,  1.23377879,  1.01321854,  2.54966184,  1.56054978,
        1.12028506,  1.15309996,  3.93138231, 16.3563883 ,  2.29907794,
        2.03259973,  1.81082676, 38.16849119,  1.04813912, 10.58750084,
        9.28336717,  1.6503357 ,  1.28397709, 31.66864823,  4.22525998,
        2.19752865,  2.2631794 ,  6.77027898,  1.42513647,  1.1069863 ,
        2.19544385,  2.46104717,  2.15963204,  1.35018114,  1.48

вычисление полученных аналитически доверительных интервалов

In [5]:
beta = 0.95
t1 = -1.96
t2 = 1.96

оценка

In [6]:
assessment_teta = (n / np.sum(np.log(data))) + 1
assessment_teta

2.0800699664863407

In [7]:
h1 = assessment_teta + t1 * (assessment_teta - 1) / n**0.5
h2 = assessment_teta + t2 * (assessment_teta - 1) / n**0.5
print(f"h1 - {h1}\nh2 - {h2}\nдлина интервала - {h2 - h1}")
an_res_teta = (h1,h2)

h1 - 1.8683762530550179
h2 - 2.2917636799176635
длина интервала - 0.42338742686264563


оценка медианы

In [8]:
assessment_med = np.power(2, 1 / (assessment_teta - 1))
assessment_med

1.8998242259467202

In [9]:
h1 = t1 * assessment_med * np.log(2) / (assessment_teta - 1) / np.sqrt(n) + assessment_med
h2 = t2 * assessment_med * np.log(2) / (assessment_teta - 1) / np.sqrt(n) + assessment_med
print(f"h1 - {h1}\nh2 - {h2}\nдлина интервала - {h2 - h1}")
an_res_med = (h1,h2)

h1 - 1.6608544018237046
h2 - 2.1387940500697358
длина интервала - 0.4779396482460312


Непараметрический бутстрап для оценки теты

In [10]:
func = lambda x: (n / np.sum(np.log(x))) + 1 - assessment_teta
non_parametric_bootstrap_teta = st.bootstrap(
    (data, ), 
    func, 
    n_resamples=1_000, 
    method='basic',
    confidence_level=0.95
).confidence_interval

non_parametric_bootstrap_teta = (assessment_teta - non_parametric_bootstrap_teta.high, assessment_teta - non_parametric_bootstrap_teta.low)
print("доверительный интервал - ",non_parametric_bootstrap_teta, "\nдлина интервала -",non_parametric_bootstrap_teta[1] - non_parametric_bootstrap_teta[0])

доверительный интервал -  (1.9193410159583437, 2.2922835277802336) 
длина интервала - 0.37294251182188987


Параметрический бутстрап для оценки теты

In [12]:
from collections.abc import Callable
from typing import NamedTuple


class ConfidenceInterval(NamedTuple):
    low: float
    high: float


def parametric_bootstrap(
    data: np.ndarray,
    statistic: Callable[[np.ndarray], float],
    prob_model: Callable[[int], np.ndarray],
    n_resamples: int = 10_000,
    confidence_level: float = 0.95,
) -> ConfidenceInterval:
    n = len(data)
    bootstrap_data = np.ndarray((n_resamples, ), dtype=np.float32)

    for i in range(n_resamples):
        resample = prob_model(n)
        bootstrap_data[i] = statistic(resample)
    
    bootstrap_data.sort()

    return ConfidenceInterval(
        low=bootstrap_data[round((1 - confidence_level) / 2 * n_resamples) - 1],
        high=bootstrap_data[round((1 + confidence_level) / 2 * n_resamples) - 1]
    )


parametric_bootstrap_teta = parametric_bootstrap(
    data,
    func,
    lambda n: np.power((1 - np.random.rand(n)), 1 / (1 - assessment_teta)),
    n_resamples=50_000, 
    confidence_level=0.95
)

parametric_bootstrap_teta = (assessment_teta - parametric_bootstrap_teta.high, assessment_teta - parametric_bootstrap_teta.low)
print("доверительный интервал - ",parametric_bootstrap_teta, "\nдлина интервала -",parametric_bootstrap_teta[1] - parametric_bootstrap_teta[0])

доверительный интервал -  (1.830410905054456, 2.264678753784543) 
длина интервала - 0.4342678487300873


Непараметрический бутстрап для оценки медианы

In [13]:
func = lambda x: np.power(2, 1 / (n / np.sum(np.log(x)))) - assessment_med
non_parametric_bootstrap_med = st.bootstrap(
    (data, ), 
    func, 
    n_resamples=1_000, 
    method='basic',
    confidence_level=0.95
).confidence_interval

non_parametric_bootstrap_med  = (assessment_med - non_parametric_bootstrap_med .high, assessment_med - non_parametric_bootstrap_med .low)
print("доверительный интервал - ",non_parametric_bootstrap_med, "\nдлина интервала -",non_parametric_bootstrap_med[1] - non_parametric_bootstrap_med[0])

доверительный интервал -  (1.6918524282943872, 2.1236498180277796) 
длина интервала - 0.43179738973339243


Параметрический бутстрап для оценки медианы

In [14]:
from collections.abc import Callable
from typing import NamedTuple


class ConfidenceInterval(NamedTuple):
    low: float
    high: float


def parametric_bootstrap(
    data: np.ndarray,
    statistic: Callable[[np.ndarray], float],
    prob_model: Callable[[int], np.ndarray],
    n_resamples: int = 10_000,
    confidence_level: float = 0.95,
) -> ConfidenceInterval:
    n = len(data)
    bootstrap_data = np.ndarray((n_resamples, ), dtype=np.float32)

    for i in range(n_resamples):
        resample = prob_model(n)
        bootstrap_data[i] = statistic(resample)
    
    bootstrap_data.sort()

    return ConfidenceInterval(
        low=bootstrap_data[round((1 - confidence_level) / 2 * n_resamples) - 1],
        high=bootstrap_data[round((1 + confidence_level) / 2 * n_resamples) - 1]
    )


parametric_bootstrap_med = parametric_bootstrap(
    data,
    func,
    lambda n: np.power(np.random.rand(n), 1 / (1 - assessment_teta)),
    n_resamples=50_000, 
    confidence_level=0.95
)

parametric_bootstrap_med = (assessment_teta - parametric_bootstrap_med.high, assessment_teta - parametric_bootstrap_med.low)
print("доверительный интервал - ",parametric_bootstrap_med, "\nдлина интервала -",parametric_bootstrap_med[1] - parametric_bootstrap_med[0])

доверительный интервал -  (1.8145261182102024, 2.294964782765752) 
длина интервала - 0.4804386645555496


Сравнение

In [15]:
print(
    "Для теты",
    f"{an_res_teta = }",
    f"{non_parametric_bootstrap_teta = }",
    f"{parametric_bootstrap_teta = }",
    "\nДля медианы",
    f"{an_res_med = }",
    f"{non_parametric_bootstrap_med = }",
    f"{parametric_bootstrap_med = }",
    sep="\n\n"
)

Для теты

an_res_teta = (1.8683762530550179, 2.2917636799176635)

non_parametric_bootstrap_teta = (1.9193410159583437, 2.2922835277802336)

parametric_bootstrap_teta = (1.830410905054456, 2.264678753784543)


Для медианы

an_res_med = (1.6608544018237046, 2.1387940500697358)

non_parametric_bootstrap_med = (1.6918524282943872, 2.1236498180277796)

parametric_bootstrap_med = (1.8145261182102024, 2.294964782765752)
